# `qcrypto` Showcase
## Setup

In [ ]:
from qcrypto.simbasics import QstateEnt, QstateUnEnt

## Introduction

This notebook presents the key features of `qcrypto`, a Python library for the simulation of simple quantum cryptography simulations. The primary classes of this library are `QstateEnt` and `QstateUnEnt`. These classes are used to represent the quantum state of a set of qubits. The former is the most general, with which one is capable of simulating the state of a possibility entangled set of qubits, whereas the latter can only be used to simulate a system of qubits which are definitely unentangled. Despite this limitation, `QstateUnEnt` offers higher performance making it ideal for simpler simulations in which the user only needs to construct a system of unentangled qubits.

The way these two classes encode the quantum state is different. 

## Unentangled State Representation

If a set of $N$ qubits are unentangled, and the $i$'th qubit's state is given by $\ket\psi_i$, then the state of the system as a whole, $\ket \psi$ can be represented as:
$$
    \ket\psi = \bigotimes_{i=0}^{N} c_i \ket\psi_i
$$
If $\psi^{(j)}_i$ represents the probability amplitude that the $i$'th qubit will be measured to be in state $\ket j$, where $j\in\{0, 1\}$, then this can be vectorially represented in the following manner:
$$
    \ket\psi =  \bigotimes_{i=0}^{N} \begin{pmatrix} \psi^{(0)}_i \\ \psi^{(1)}_i \end{pmatrix}
$$
`QstateUnEnt` takes advantaged of the separability of the state of each individual qubit by representing the state of the system as a whole as a $N \times 2$ dimensional numpy array where each row corresponds to a qubit and each column the probability amplitude of each possible state of each individual qubit. Moreover, for simplicity, each qubit state is normalized separately.

In [ ]:
# Unentangled set of n qubits
N = 5
qstateunent = QstateUnEnt(init_method="random", num_qubits=N)
qstateunent

## Entangled State Representation

In the case of a (possibly) entangled set of qubits, `qcrypto` offers the `QstateEnt` class. For entangled qubits it is not possible to represent the state of the system as a whole as a tensor product of the state of each individual qubit. Suppose we have $N$ qubits. In this case, we would represent the state of the system as:
$$
    \ket \psi = \sum_{x=0}^{2^N -1} c_x \ket{x_1 x_2 \dots x_N}
$$

where $x_i\in\{0,1\}$. This means that a system of 2 qubits would be represented as:

$$
    \ket\psi = c_{0}\ket{00} + c_{1}\ket{01} + c_{2}\ket{10} + c_{3} \ket{11}
$$

`QstateEnt` represents such states as a 1D numpy array of length $2^N$.

In [ ]:
# Entangled set of n qubits
N = 5
qstateent = QstateEnt(init_method="random", num_qubits=N)
qstateent

## Measurements

In order to measure a qubit, one can call the `measure` method, which requires the specification of which qubit is to measured, or the `measure_all` method, which measures all of the qubits. For `QstateUnEnt`, `measure_all` simply applies `measure` to all qubits sequentially. However, for `QstateEnt`, because the order of measurement is important, requires the user to specify if the system should be measured simultaneously (all qubits measured at one), or sequentially.

When measurements are made, the state of the system is updated in order to reflect the collapse of the quantum state of the system and the method returns the result of the measurement. We can observe that in the following cells.

In [ ]:
"""
Measurement of one unentangled qubits
Because the states are considered separately, only the qubit that was measured has it state updated.
"""
msrmt_rslt = qstateunent.measure(qubit_idx=0)
print(f"Result from measurement: {msrmt_rslt}")
print("Updated state:\n", qstateunent)

In [ ]:
"""
Measurement of all unentangled qubits
"""
all_msrmt_rslt = qstateunent.measure_all()
print(f"Result from measurement: {all_msrmt_rslt}")
print("Updated state:\n", qstateunent)

In [ ]:
"""
Measurement of one entangled qubits
Because the qubits are (possibly) entangled, the post-measurement update is applied to all states.
"""
msrmnt_rslt_ent = qstateent.measure(qubit_idx=0)
print(f"Result from measurement: {msrmnt_rslt_ent}")
print("Updated state:\n", qstateent)

In [ ]:
"""
Measurement of all entangled qubits
"""
all_msrmt_rslt_ent = qstateent.measure_all(order="simult")
print(f"Result from measurement: {all_msrmt_rslt_ent}")
print("Updated state:\n", qstateent)

## Gates

`qcrypto`'s quantum state classes also include a method named `apply_gate` which, as the name suggests, allows the user to apply a qunatum gate to one or more of the qubits. There are two gates included in the library under the module `qcrypto.gates`, namely the Hadamard and Pauli gates, but more can defined by the user by simply construction an appropriate 2D numpy array.

As an example of the application of gates, we can apply the Hadamard gate which is given by
$$
    H = \frac{1}{\sqrt{2}}\begin{pmatrix}
        1 &  1\\
        1 & -1\\
    \end{pmatrix}
$$

In [ ]:
from qcrypto.gates import H_gate

In [ ]:
"""
Application the H gate to the unentangled qubits
Note that, given the limitations of this class in only being able to represent unentangled sets,
gates which create entangled states will not work. 
"""
print(qstateunent)
qstateunent.apply_gate(H_gate)
print(f"H * state = \n {qstateunent}")
qstateunent.apply_gate(H_gate)
print(f"H * H * state = \n {qstateunent}")

In [ ]:
qubit_idx = 2
qstateunent.apply_gate(H_gate, qubit_idx)
print(f"H * state_{qubit_idx} = \n {qstateunent}")


In [ ]:
qubit_idx = [0, 3]
qstateunent.apply_gate(H_gate, qubit_idx)
print(f"H * state_{qubit_idx} = \n {qstateunent}")

In [ ]:
"""
Application the H gate to the entangled qubits
Because the qubits are entangled, only the application of gates to the system as a whole is permitted.
"""
print(f"state = \n {qstateent}")
qstateent.apply_gate(H_gate)
print(f"H * state = \n {qstateent}")

## Agent
In order to represent the "player" in quantum cryptography simulations, `qcrypto` includes the class `Agent`. In summary, this class:
* Can store two `Qstate` objects, one intended to be public (i.e. accessible by other agents), and one private (i.e. meant to only be accessible by the agent in its possession)
* Can generate a public/private keys through the measurement of the public/private `Qstate` and is able to store them
* Can apply gates the public/private quantum states
* Can measure individual public/private qubits

In order to properly showcase its functionality, please refer to the `BB84.ipynb` file which includes the simulation of the BB84 quantum key distribution scheme using this class. 